# Evaluación de modelos de texto generativo

Se comenzará la sección configurando el LLM para la generación de texto a partir del código implementado en la sección 3.

![Texto alternativo](./imgs/5.2.png)

La  siguiente  subsección  resume  la  generación  de  texto  que se configuró al final de la sección 3, antes  de  sumergirnos  en  la  evaluación  del  texto  y  el  cálculo  
de  las  pérdidas  de  entrenamiento  y  validación  en  las  subsecciones  posteriores.

## Uso de GPT para generar texto

En esta subsección se configura el LLm y se recapitula brevemente el proceso de generación de texto

In [ ]:
from seccion04_ImplementacionGPTGeneracionTexto import gptModel

ModuleNotFoundError: No module named 'transformeBlock'